In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count, desc, max, min

In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

23/11/14 16:54:37 WARN Utils: Your hostname, Pradyoths-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.18.11.34 instead (on interface en0)
23/11/14 16:54:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/14 16:54:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
orc_data = spark.read.format("orc").load("kafka_output/")


In [4]:
cleaned_data = orc_data.na.drop()


### 1. Printing top category_brand combination for each user.


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, col
import matplotlib.pyplot as plt

# Initialize Spark session
spark = SparkSession.builder.appName("user_category_interaction").getOrCreate()

# Assuming 'cleaned_data' is your existing Spark DataFrame

# Group by 'user_id', 'category_code', count interactions
user_category_interaction_count = cleaned_data.groupBy('user_id', 'category_code').count()

# Sort by count in descending order
user_category_interaction_count = user_category_interaction_count.orderBy(desc('count'))

# Collect the top 10 interactions
top_10_interactions = user_category_interaction_count.limit(10).toPandas()

print(top_10_interactions)

# Save the top 10 interactions to CSV format
# top_10_interactions.to_csv('spark_output/top_10_interactions.csv', index=False)

# # Plotting the top 10 user interactions
# plt.figure(figsize=(10, 6))

# # Group by 'category_code', collect counts, and plot
# category_counts = top_10_interactions.groupBy('category_code').agg({'count': 'collect_list'}).collect(

# plt.figure(figsize=(10, 6))
# for category in category_counts:
#     category_code = category['category_code']
#     counts = category['collect_list(count)']
#     plt.bar([str(i) for i in range(len(counts))], counts, label=category_code)

# plt.xlabel('User ID')
# plt.ylabel('Count')
# plt.title('User Interaction Counts for Top 10 Categories')
# plt.legend()
# plt.tight_layout()
# plt.show()


# # Collect the top 10 interactions
# top_10_interactions = top_10_interactions.toPandas()

# # Save the top 10 interactions to CSV format
# top_10_interactions.to_csv('spark_output/top_10_interactions.csv', index=False)


     user_id            category_code  count
0  569335945            kids.carriage  13647
1  568778435                           12557
2  512365995   electronics.smartphone   9430
3  512505687   electronics.smartphone   5562
4  514649263            kids.carriage   5558
5  559249905                            5385
6  512388419   electronics.smartphone   5331
7  513021392   electronics.smartphone   4390
8  512475445  auto.accessories.player   4303
9  537873067                            4169


### 2. Most Engaged Users within Categories and Brands:
Identify users who interact the most with specific categories or brands.

In [17]:
# Initialize Spark session
spark = SparkSession.builder.appName("Most_Engaged_Users").getOrCreate()

# Assuming 'orc_data' is your existing Spark DataFrame

# Group by 'user_id', 'category_code', 'brand' and count interactions
user_category_brand_interaction_count = cleaned_data.groupBy('user_id', 'category_code', 'brand').count()

# Find the maximum count for each category and brand
max_interactions_per_category_brand = user_category_brand_interaction_count.groupBy('category_code', 'brand') \
    .agg(max('count').alias('max_count'))

# Join to find the most engaged users for each category and brand
most_engaged_users = user_category_brand_interaction_count.join(max_interactions_per_category_brand,
                                                               on=['category_code', 'brand'], how='inner')

# Show the most engaged users within each category and brand in descending order of interaction counts
print(most_engaged_users.select('category_code', 'brand', 'user_id', 'max_count') \
    .orderBy('max_count', ascending=False).show())
print(most_engaged_users.select('category_code', 'brand', 'user_id', 'max_count') \
    .orderBy('max_count', ascending=True).show())

23/11/13 21:44:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/11/13 21:48:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:48:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:48:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:48:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:48:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:48:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:48:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:48:43 WARN RowBasedKeyValueBatch: Calling spill()

+--------------------+-------+---------+---------+
|       category_code|  brand|  user_id|max_count|
+--------------------+-------+---------+---------+
|electronics.smart...|samsung|513805641|     5597|
|electronics.smart...|samsung|548680744|     5597|
|electronics.smart...|samsung|559687595|     5597|
|electronics.smart...|samsung|555271034|     5597|
|electronics.smart...|samsung|514652411|     5597|
|electronics.smart...|samsung|549264178|     5597|
|electronics.smart...|samsung|518873554|     5597|
|electronics.smart...|samsung|513115532|     5597|
|electronics.smart...|samsung|521773470|     5597|
|electronics.smart...|samsung|534538688|     5597|
|electronics.smart...|samsung|513816945|     5597|
|electronics.smart...|samsung|556736963|     5597|
|electronics.smart...|samsung|551047678|     5597|
|electronics.smart...|samsung|522742516|     5597|
|electronics.smart...|samsung|512819219|     5597|
|electronics.smart...|samsung|541760697|     5597|
|electronics.smart...|samsung|5

23/11/13 21:55:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:55:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:55:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:55:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:55:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:55:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:55:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:55:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/13 21:55:18 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+------------+---------+---------+
|       category_code|       brand|  user_id|max_count|
+--------------------+------------+---------+---------+
|                    |    elkitorg|518334158|        1|
|                    |     bioline|539320069|        1|
|                    |      casato|570455936|        1|
|                    |     bioline|518049639|        1|
|                    |        dabo|565727446|        1|
|                    |     bioline|518921498|        1|
|apparel.shoes.bal...|       baden|516497044|        1|
|                    |   samsonite|577452466|        1|
|                    |      aufine|515906876|        1|
|                    |   samsonite|515986787|        1|
|                    |    ecocraft|512432515|        1|
|                    |   samsonite|522819804|        1|
|                    |       leeco|545656741|        1|
|                    |   samsonite|535729498|        1|
|                    |    ecocraft|520451797|   

In [ ]:
#### Price Sensitivity within Categories: Evaluate user interactions with varying price ranges within different categories. This could reveal if users tend to engage more with specific price ranges or if certain categories are associated with higher-priced items.
##### This code will group the data by 'category_code' and calculate the average, maximum, and minimum prices within each category. It provides insights into how users interact with varying price ranges within different categories, offering an understanding of price sensitivity across categories. Adjustments or further analysis can be performed based on your specific requirements.

In [11]:
# Assuming 'cleaned_data' DataFrame is available and 'avg_prices' calculation is already done
spark = SparkSession.builder.appName("Average").getOrCreate()

price_sensitivity = cleaned_data.groupBy('category_code').agg(
    avg('price').alias('average_price'),
    max('price').alias('max_price'),
    min('price').alias('min_price')
)

# Get top 10 categories by average price
top_10_avg_price_categories = price_sensitivity.orderBy(desc('average_price')).limit(10).toPandas()

print(top_10_avg_price_categories)

# # Collect data for plotting
# top_10_categories = top_10_avg_price_categories['category_code']
# top_10_avg_prices = top_10_avg_price_categories['average_price']
# top_10_max_prices = top_10_avg_price_categories['max_price']
# top_10_min_prices = top_10_avg_price_categories['min_price']

# # Creating a line plot for average prices per category for top 10 categories
# plt.figure(figsize=(10, 6))
# plt.plot(top_10_categories, top_10_avg_prices, marker='o', linestyle='-', color='green', label='Average Price')
# plt.plot(top_10_categories, top_10_max_prices, marker='o', linestyle='-', color='red', label='Max Price')
# plt.plot(top_10_categories, top_10_min_prices, marker='o', linestyle='-', color='blue', label='Min Price')

# plt.xlabel('Category Code')
# plt.ylabel('Price')
# plt.title('Price Sensitivity for Top 10 Categories')
# plt.xticks(rotation=45)
# plt.legend()
# plt.tight_layout()

# # Show the plot
# plt.show()

# # Save the top 10 categories to a CSV
# top_10_avg_price_categories.to_csv('spark_output/top_10_categories_prices.csv', index=False)

23/11/14 17:14:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


                   category_code  average_price  max_price  min_price
0       electronics.camera.photo     807.097387    2567.05       0.00
1             computers.notebook     719.644028    2574.04       0.00
2    electronics.video.projector     705.805075    2433.34       0.00
3     furniture.living_room.sofa     622.351714    2574.04       0.00
4              computers.desktop     579.218956    2574.04       0.00
5         electronics.smartphone     479.427674    2562.49       0.00
6         auto.accessories.winch     474.437349     984.84     221.88
7           electronics.video.tv     460.377066    2574.04       0.00
8  appliances.kitchen.dishwasher     425.146637    2085.57       0.00
9                  sport.bicycle     419.814617    2573.81       0.00


23/11/14 19:20:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 345224 ms exceeds timeout 120000 ms
23/11/14 19:20:09 WARN SparkContext: Killing executors is not supported by current scheduler.
23/11/14 19:20:09 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc, col

# Initialize Spark session
spark = SparkSession.builder.appName("Popular_Categories").getOrCreate()

## Group by 'category_id' and count interactions
popular_categories = cleaned_data.groupBy('category_id','category_code').agg(count('*').alias('interaction_count'))

# Sort by count in descending order
popular_categories = popular_categories.orderBy(desc('interaction_count'))

# Show the most frequently interacted categories
popular_categories.show()


23/11/13 22:56:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------------------+--------------------+-----------------+
|        category_id|       category_code|interaction_count|
+-------------------+--------------------+-----------------+
|2053013555631882655|electronics.smart...|         28833250|
|2053013553559896355|                    |          5294285|
|2053013558920217191|  computers.notebook|          3394767|
|2053013554415534427|electronics.video.tv|          3374747|
|2053013554658804075|electronics.audio...|          3007758|
|2053013563810775923|appliances.kitche...|          2329638|
|2053013565983425517|appliances.enviro...|          2328665|
|2053013563651392361|                    |          2303605|
|2053013553341792533|  electronics.clocks|          1882651|
|2053013561579406073|  electronics.clocks|          1630396|
|2053013563911439225|appliances.kitche...|          1615724|
|2053013563693335403|                    |          1439352|
|2053013556168753601|                    |          1436253|
|2053013563970159485|   

In [ ]:
local_popular_categories = popular_categories.limit(10).toPandas()
local_popular_categories.to_csv('spark_output/top_10_categories.csv', index=False)


# Plotting a pie chart using Matplotlib
plt.figure(figsize=(8, 8))
plt.pie(local_popular_categories['interaction_count'], labels=local_popular_categories['category_code'], autopct='%1.1f%%')
plt.title('Most Frequently Interacted Categories')
plt.axis('equal')  # Equal aspect ratio ensures the pie is drawn as a circle.
plt.show()